In [1]:
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
from unidecode import unidecode
from fuzzywuzzy import process

import os
import zipfile
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import Levenshtein
import datetime

In [2]:
df_R1_matricula = pd.read_csv("../../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Consolidado_4_mayo_Matricula.csv", encoding = "latin1")
df_R1_matricula["CODIGO_ORIP"] = df_R1_matricula["CODIGO_ORIP"].astype(str).str.replace(".0", "")

df_R1_matricula["MUNICIPIO"] = df_R1_matricula["MUNICIPIO"].astype(int).apply(round).astype(str).str.zfill(3)
df_R1_matricula["DEPARTAMENTO"] = df_R1_matricula["DEPARTAMENTO"].astype(int).apply(round).astype(str).str.zfill(2)
df_R1_matricula["DIVIPOLA"] = df_R1_matricula["DEPARTAMENTO"] + df_R1_matricula["MUNICIPIO"]
#ofertas_final["Municipio_x"] = ofertas_final["Municipio_x"].astype(int).apply(round).astype(str).str.zfill(3)
#ofertas_final["Departamento_x"] = ofertas_final["Departamento_x"].astype(int).apply(round).astype(str).str.zfill(2)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_6656\3658018204.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_R1_matricula = pd.read_csv("../../230628_COMPARACION_R1_DIFERENTES_FUENTES/output/230703_R1_Consolidado_4_mayo_Matricula.csv", encoding = "latin1")


In [3]:
# df_matricula_R1 = pd.read_csv("../input/20230526_R1_Matricula.csv")
# df_matricula_R1_v2 = df_matricula_R1[df_matricula_R1["MATRICULA_INMOBILIARIA"].notnull()][["NPN", "MATRICULA_INMOBILIARIA", "NUMERO_DEL_PREDIO", 'Departamento', 'Municipio', "DEPARTAMENTO", "DIVIPOLA"]]
# df_matricula_R1_v2 = df_matricula_R1_v2.rename(columns={"NPN" : "NPN_R1", "MATRICULA_INMOBILIARIA" : "MATRICULA_INMOBILIARIA_R1", "NUMERO_DEL_PREDIO" : "NUMERO_DEL_PREDIO_R1", 'Departamento' : 'Departamento_R1', 'Municipio' : 'Municipio_R1', "DEPARTAMENTO" : "CODIGO_DEPARTAMENTO_R1", "DIVIPOLA" : "DIVIPOLA_R1"})
# df_matricula_R1_v2.columns
# #df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1_v2"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
# df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"] = df_matricula_R1_v2["MATRICULA_INMOBILIARIA_R1"].str.strip()
# pattern = r'[^a-zA-Z0-9]'

#df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'] = df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1_v2'].astype(str).str.replace(pattern, '', regex = True)

# df_matricula_R1_v3 = df_matricula_R1_v2[df_matricula_R1_v2['MATRICULA_INMOBILIARIA_R1'].str.count("-") == 1]
# df_matricula_R1_v3['Matricula'] = df_matricula_R1_v3['MATRICULA_INMOBILIARIA_R1'].str.extract(r'-(\d+)')
# df_matricula_R1_v3['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_matricula_R1_v3['DIVIPOLA_R1'].astype(int).apply(round).astype(str).str.strip() + "-" + df_matricula_R1_v3['Matricula'].astype(str).str.strip()

In [4]:
divipola = pd.read_excel("../input/CODIGOS_DIVIPOLA.xlsx", "Listado Vigentes")

divipola["Nombre Municipio"] = divipola["Nombre Municipio"].astype(str).str.strip().apply(lambda x: unidecode(x))
divipola["Nombre Departamento"] = divipola["Nombre Departamento"].astype(str).str.strip().apply(lambda x: unidecode(x))

divipola["Código Municipio"] = divipola["Código Municipio"].astype(int).apply(round)
divipola["Código Departamento"] = divipola["Código Departamento"].astype(int).apply(round)

In [5]:
ofertas_final = pd.read_csv("../output/230725_RESULTADOS_GEOREF_SUCRE.csv")
ofertas_final.columns
ofertas_final.drop(['Direccion_y', 'Municipio_y','Departamento_y', 'Código Departamento', 'Código Municipio_y', 'Código Departamento', 'Código Municipio_y', 'Código Municipio_x', 'Nombre Municipio', 'Nombre Departamento'], axis = 1, inplace=True)
ofertas_final.rename({"Direccion_x": "Direccion", "Departamento_x" : "Departamento", "Municipio_x" : "Municipio"}, inplace = True, axis = 1)
ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "DIVIPOLA"] = ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "Municipio"]
ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "DIVIPOLA"] = ofertas_final.loc[ofertas_final["DIVIPOLA"].isnull(), "Municipio"]
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(str).str.replace("\.0+", "", regex = True)
ofertas_final.loc[ofertas_final["DIVIPOLA"].str.contains("\D", regex = True), "DIVIPOLA"] = None
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].fillna(0)
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(int).apply(round).astype(str).str.zfill(5)
ofertas_final["Municipio"] = ofertas_final["DIVIPOLA"]
ofertas_final['Departamento'] = ofertas_final['DIVIPOLA'].apply(lambda x: x[:2])

In [6]:
divipola["Código Municipio"] = divipola["Código Municipio"].astype(int).apply(round).astype(str).str.zfill(5)
ofertas_final = pd.merge(ofertas_final, divipola[["Código Municipio", "Nombre Municipio","Nombre Departamento"]].drop_duplicates(), left_on = "DIVIPOLA", right_on = "Código Municipio", how = "left")
ofertas_final["Municipio_nombre"] = ofertas_final["Nombre Municipio"]
ofertas_final["Departamento_nombre"] = ofertas_final["Nombre Departamento"]
ofertas_final.drop(["Código Municipio", "Nombre Municipio","Nombre Departamento"], axis = 1, inplace = True)

In [7]:
tabla_codigos_orip = pd.read_excel("../input/tablaORIP.xlsx", sheet_name = "Hoja1")

#tabla_codigos_orip["CÓDIGO ORIP"] = tabla_codigos_orip["CÓDIGO ORIP"].str.strip().astype(str)
tabla_codigos_orip["DIVIPOLA"] = tabla_codigos_orip["DIVIPOLA"].fillna(0)
tabla_codigos_orip["DIVIPOLA"] = tabla_codigos_orip["DIVIPOLA"].astype(str).str.replace(".0", "").str.zfill(5)
tabla_codigos_orip[tabla_codigos_orip["DIVIPOLA"].astype(str).str.contains("25168")]

,CÓDIGO ORIP,DEPARTAMENTO,COBERTURA,DIVIPOLA,NOMBRE OFICINA DE REGISTRO,NOMBRE ORIP,FECHA DE CREACIÓN,ACTO ADMINISTRATIVO DE CREACIÓN,JEFE DE OFICINA,CORREO JEFE,CONTACTO
603,162,CUNDINAMARCA,CHAGUANÍ,25168,GUADUAS,NaN,NaN,NaN,DAVID RAMIREZ GUERRA,david.ramirez@supernotariado.gov.co,(1) 8417430 (1)8466266


In [8]:
codigos_orip_divipola = tabla_codigos_orip[["DIVIPOLA", "CÓDIGO ORIP"]].drop_duplicates("DIVIPOLA")
df_R1_matricula["DIVIPOLA"] = df_R1_matricula["DIVIPOLA"].astype(str)
ofertas_final["DIVIPOLA"] = ofertas_final["DIVIPOLA"].astype(str)
codigos_orip_divipola["DIVIPOLA"] = codigos_orip_divipola["DIVIPOLA"].astype(str)
df_R1_matricula = pd.merge(df_R1_matricula, codigos_orip_divipola, how = "left", on = "DIVIPOLA")
ofertas_final = pd.merge(ofertas_final, codigos_orip_divipola, how = "left", on = "DIVIPOLA")

In [9]:
replacement_string = ''

prefixes_to_replace = ['347', '342', '340', '346']
prefixes_to_replace_2 = ["MI","HA","HC","-ACLA"]

# Use the apply method to conditionally replace the strings
ofertas_final['Matricula_Inmobiliaria'] = ofertas_final['Matricula_Inmobiliaria'].astype(str)
ofertas_final['Matricula_Inmobiliaria'] = ofertas_final['Matricula_Inmobiliaria'].apply(lambda x: replacement_string + x[3:] if any(x.startswith(prefix) for prefix in prefixes_to_replace_2) else x)
ofertas_final['Matricula_Inmobiliaria'] = ofertas_final['Matricula_Inmobiliaria'].apply(lambda x: replacement_string + x[3:] if any(x.startswith(prefix) for prefix in prefixes_to_replace) else x)

df_matriculas = pd.DataFrame({"Matricula_Inmobiliaria" : ofertas_final["Matricula_Inmobiliaria"].str.strip().str.extractall(r'(\d+)')[0].reset_index(level=1, drop=True)})
df_matriculas.tail()

df_merge = pd.merge(df_matriculas, ofertas_final[["CÓDIGO ORIP"]], how = "left", left_index = True, right_index = True)

df_merge["Matricula_Divipola"] = df_merge["CÓDIGO ORIP"].astype(str) + "-" + df_merge["Matricula_Inmobiliaria"].astype(str)
#ofertas_final[["DIVIPOLA"]]

In [10]:
#df_merge[["Matricula_Inmobiliaria", "CÓDIGO ORIP"]]
#ofertas_final[ofertas_final["DIVIPOLA"] != "70001"][["Matricula_Inmobiliaria", "CÓDIGO ORIP"]].iloc[120:150]
ofertas_final[ofertas_final["DIVIPOLA"] != "70001"]["CÓDIGO ORIP"].value_counts()

CÓDIGO ORIP
347    284
342    261
340    179
346     11
Name: count, dtype: int64

In [11]:
df_R1_matricula["MATRICULA"].fillna(0, inplace=True)
df_R1_matricula = df_R1_matricula[df_R1_matricula["NUMERO_DE_ORDEN"].isin([1, "1"])]
df_R1_matricula["MATRICULA"] = df_R1_matricula["MATRICULA"].astype(str).str.replace(".0", "").str.strip()
df_R1_matricula["NPN_R1"] = df_R1_matricula["DEPARTAMENTO"] + df_R1_matricula["MUNICIPIO"] + df_R1_matricula["NUMERO_DEL_PREDIO"]
#df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_R1_matricula['CÓDIGO ORIP'].astype(str).str.strip() + "-" + df_R1_matricula['MATRICULA'].astype(str).str.strip()
#df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'] = df_R1_matricula['MATRICULA_INMOBILIARIA_DIVIPOLA_R1'].str.strip().str.replace(" ", "").astype(str)
df_merge['Matricula_Divipola'] = df_merge['Matricula_Divipola'].str.strip().str.replace(" ", "").astype(str)
df_merge.reset_index(inplace= True)
df_merge_2 = pd.merge(df_merge, df_R1_matricula[["CÓDIGO ORIP", "MATRICULA", "DEPARTAMENTO", "MUNICIPIO", "NUMERO_DEL_PREDIO", "NPN_R1"]].drop_duplicates(), how = "inner", left_on = ["CÓDIGO ORIP", "Matricula_Inmobiliaria"], right_on = ["CÓDIGO ORIP", "MATRICULA"])
df_merge_2.drop_duplicates("index",inplace= True)
#df_merge_2.set_index("index", inplace= True)
df_merge_2.shape
	

(609, 9)

In [12]:
ORIP_UNIQUE_CODES = df_merge[df_merge["CÓDIGO ORIP"].isin(df_R1_matricula["CÓDIGO ORIP"])]["CÓDIGO ORIP"].unique()

df_aprox_matriculas_final = []
for orip in ORIP_UNIQUE_CODES:
    orip = ORIP_UNIQUE_CODES[0]
    df_avaluos = df_merge[df_merge["CÓDIGO ORIP"].astype(str) == str(orip)].copy()
    df_matricula_orip = df_R1_matricula[df_R1_matricula["CÓDIGO ORIP"].astype(str) == str(orip)].copy()

    df_avaluos = df_avaluos[~df_avaluos["Matricula_Inmobiliaria"].str.contains("\D", regex = True)]
    df_matricula_orip = df_matricula_orip[["CÓDIGO ORIP", "MATRICULA", "DEPARTAMENTO", "MUNICIPIO", "NUMERO_DEL_PREDIO", "NPN_R1"]].drop_duplicates()
    df_matricula_orip = df_matricula_orip[~df_matricula_orip["MATRICULA"].str.contains("\D", regex = True)]

    #df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype(int)

    #df_avaluos.sort_values("Matricula_Inmobiliaria")
    #df_avaluos
    #df_matricula_orip[["MATRICULA"]]
    df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype("Int64")
    df_matricula_orip["MATRICULA"] = df_matricula_orip["MATRICULA"].astype("Int64")
    df_avaluos["Matricula_Inmobiliaria"] = df_avaluos["Matricula_Inmobiliaria"].astype("int")
    df_matricula_orip["MATRICULA"] = df_matricula_orip["MATRICULA"].astype("int")
    df_avaluos.sort_values("Matricula_Inmobiliaria", inplace=True)
    df_matricula_orip.sort_values("MATRICULA", inplace=True)

    df_avaluos_matricula_aprox = pd.merge_asof(df_avaluos, df_matricula_orip[["MATRICULA","NPN_R1"]], left_on = "Matricula_Inmobiliaria", right_on = "MATRICULA")

    # Function to calculate Levenshtein distance
    def calculate_distance(row):
        return Levenshtein.distance(row['Matricula_Inmobiliaria'], row['MATRICULA'])

    df_avaluos_matricula_aprox["Matricula_Inmobiliaria"] = df_avaluos_matricula_aprox["Matricula_Inmobiliaria"].astype(str)
    df_avaluos_matricula_aprox["MATRICULA"] = df_avaluos_matricula_aprox["MATRICULA"].astype(str)
    # Apply the function to each row and store the results in a new column
    df_avaluos_matricula_aprox['distance'] = df_avaluos_matricula_aprox.apply(calculate_distance, axis=1)
    df_avaluos_matricula_aprox['Longitud_matricula'] = df_avaluos_matricula_aprox["Matricula_Inmobiliaria"].apply(len)
    df_aprox_matriculas_final.append(df_avaluos_matricula_aprox)
    #df_avaluos_matricula_aprox.tail()


In [13]:
df_aprox_matriculas_final_2 = pd.concat(df_aprox_matriculas_final, axis = 0)
df_aprox_matriculas_final_3 = df_aprox_matriculas_final_2[(df_aprox_matriculas_final_2["distance"] >= 1) & (df_aprox_matriculas_final_2["distance"] < 2) & (df_aprox_matriculas_final_2["Longitud_matricula"] > 4)]
df_aprox_matriculas_final_3["MATRICULA_APROXIMADA"] = df_aprox_matriculas_final_3["CÓDIGO ORIP"].astype(str) + "-" + df_aprox_matriculas_final_3["MATRICULA"].astype(str)
df_aprox_matriculas_final_4 = df_aprox_matriculas_final_3[["index", "MATRICULA_APROXIMADA", "NPN_R1"]]
df_aprox_matriculas_final_4.rename({"NPN_R1": "NPN_R1_APROX"}, axis = 1, inplace=True)
df_aprox_matriculas_final_4.set_index("index", inplace=True)
df_aprox_matriculas_final_4.head()


C:\Users\Administrador\AppData\Local\Temp\ipykernel_6656\4173479554.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_3["MATRICULA_APROXIMADA"] = df_aprox_matriculas_final_3["CÓDIGO ORIP"].astype(str) + "-" + df_aprox_matriculas_final_3["MATRICULA"].astype(str)
C:\Users\Administrador\AppData\Local\Temp\ipykernel_6656\4173479554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_4.rename({"NPN_R1": "NPN_R1_APROX"}, axis = 1, inplace=True)


,MATRICULA_APROXIMADA,NPN_R1_APROX
index,,
1299,340-10183,702650001000000040095000000000
1309,340-10230,708200100000000530022000000000
551,340-10230,708200100000000530022000000000
1137,340-10461,706700100000000500002000000000
923,340-10557,707130100000000780002000000000


In [14]:
df_aprox_matriculas_final_3.head()

,index,Matricula_Inmobiliaria,CÓDIGO ORIP,Matricula_Divipola,MATRICULA,NPN_R1,distance,Longitud_matricula,MATRICULA_APROXIMADA
107,1299,10186,340,340-10186,10183,702650001000000040095000000000,1,5,340-10183
108,1309,10237,340,340-10237,10230,708200100000000530022000000000,1,5,340-10230
109,551,10237,340,340-10237,10230,708200100000000530022000000000,1,5,340-10230
111,1137,10469,340,340-10469,10461,706700100000000500002000000000,1,5,340-10461
112,923,10559,340,340-000000010559,10557,707130100000000780002000000000,1,5,340-10557


In [15]:
df_merge_2.set_index("index", inplace = True)
df_merge_2["FUENTE_PROCESAMIENTO"] = "R1_MATRICULA"

In [16]:
df_aprox_matriculas_final_4.reset_index(inplace=True)
df_aprox_matriculas_final_4.drop_duplicates(inplace=True)
df_aprox_matriculas_final_4.set_index("index", inplace=True)


C:\Users\Administrador\AppData\Local\Temp\ipykernel_6656\2358486246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aprox_matriculas_final_4.drop_duplicates(inplace=True)


In [17]:
df_merge_2.head()

,Matricula_Inmobiliaria,CÓDIGO ORIP,Matricula_Divipola,MATRICULA,DEPARTAMENTO,MUNICIPIO,NUMERO_DEL_PREDIO,NPN_R1,FUENTE_PROCESAMIENTO
index,,,,,,,,,
9,68545,340,340-68545,68545,70,823,0003000000010422000000000,708230003000000010422000000000,R1_MATRICULA
10,4679,340,340-4679,4679,70,820,0100000001110011000000000,708200100000001110011000000000,R1_MATRICULA
157,101627,340,340-101627,101627,70,265,0100000001330001000000000,702650100000001330001000000000,R1_MATRICULA
229,137907,340,340-137907,137907,70,820,0102000000080024901020001,708200102000000080024901020001,R1_MATRICULA
319,137907,340,340-137907,137907,70,820,0102000000080024901020001,708200102000000080024901020001,R1_MATRICULA


In [18]:
ofertas_final_v1 = pd.merge(ofertas_final, df_merge_2[["NPN_R1", "FUENTE_PROCESAMIENTO", "Matricula_Divipola"]], how = "left", left_index = True, right_index = True)
ofertas_final_v2 = pd.merge(ofertas_final_v1,  df_aprox_matriculas_final_4, how = "left", left_index = True, right_index = True)
ofertas_final_v2.shape
#pd.merge

(3736, 60)

In [19]:

# # Use the str.contains method to filter and keep only strings that contain letters
# df = ofertas_final_v2[ofertas_final_v2['Matricula_Inmobiliaria'].str.contains('[a-zA-Z]')]
# df['extracted_characters'] = df['Matricula_Inmobiliaria'].str.extract(r'([a-zA-Z]+)')
# df.iloc[40:60]["Matricula_Inmobiliaria"]
ofertas_final_v2[(ofertas_final_v2["FUENTE_PROCESAMIENTO"] != "R1_MATRICULA") & (ofertas_final_v2["DIVIPOLA"] != "70001")].shape

(157, 60)

In [20]:
# quantiles = list(np.arange(0, 0.95, 0.05))#[0.05, 0.1, 0.2, 0.25, 0.50, 0.75, 0.90, 0.95, 0.97]
# ofertas_final_v2[(ofertas_final_v2["ZONA"] == "RURAL_APROX") & (ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()



In [21]:
pd.options.display.float_format = '{:.6f}'.format
# ofertas_final_v2[(ofertas_final_v2["ZONA"] == "URBANA_APROX") & (ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "R1_MATRICULA")]["Distancia_final"].quantile(quantiles).to_frame().reset_index()

In [22]:
ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "CODIGO_PREDIAL"] = ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "NPN_R1"]
#ofertas_final_v2.loc[ofertas_final_v2["NPN_R1"].notnull(), "Fuente"] = "OIC"
ofertas_final_v2.loc[(ofertas_final_v2["NPN_R1"].isnull()) & (ofertas_final_v2["CODIGO_PREDIAL"].notnull()), "FUENTE_PROCESAMIENTO"] = "Subdireccion"

ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.000092) & (ofertas_final_v2["ZONA"] == "URBANA_APROX"), "TIPO_GEORREF"] = "APROX_OK"
ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.008925) & (ofertas_final_v2["ZONA"] == "RURAL_APPROX"), "TIPO_GEORREF"] = "APROX_OK"

# ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.000045) & (ofertas_final_v2["ZONA"] == "URBANA_APROX"), "TIPO_GEORREF"] = "APROX_OK"
# ofertas_final_v2.loc[(ofertas_final_v2["Distancia_final"] <= 0.007404) & (ofertas_final_v2["ZONA"] == "RURAL_APPROX"), "TIPO_GEORREF"] = "APROX_OK"


In [23]:
# Get today's date
today = datetime.date.today()
# Format today's date as YYMMDD
formatted_date = today.strftime("%y%m%d")

ofertas_final_v2["SUBSTRING_67"] = ofertas_final_v2["CODIGO_PREDIAL"].str[5:7]
ofertas_final_v2["SUBSTRING_22"] = ofertas_final_v2["CODIGO_PREDIAL"].str[21:22]

# Define the conditions and corresponding values
conditions = [
    ofertas_final_v2['SUBSTRING_67'] == "00",
    ofertas_final_v2['SUBSTRING_67'] == "01",
    (~ofertas_final_v2['SUBSTRING_67'].isin(["01", "00"])) & (ofertas_final_v2['SUBSTRING_67'].notnull())
]
choices = ['RURAL', 'URBANO', 'CENTRO_P']

# Apply the case when statement using numpy.select()
ofertas_final_v2['ZONA'] = np.select(conditions, choices, default='SIN_ASIGNAR')

conditions_tipo_predio = [
    ofertas_final_v2['SUBSTRING_22'] == "9",
    ofertas_final_v2['SUBSTRING_22'] == "8",
    ofertas_final_v2['SUBSTRING_22'] == "7",
    ofertas_final_v2['SUBSTRING_22'] == "5",
    ofertas_final_v2['SUBSTRING_22'] == "4",
    ofertas_final_v2['SUBSTRING_22'] == "3",
    ofertas_final_v2['SUBSTRING_22'] == "0"    
]
choices_tipo_predio = ["PH", "CONDOMINIO", "CEMENTERIOS", "MEJORA", "VIAS", "USO_PUBLICO", "NPH"]


ofertas_final_v2['CLASE_PREDIO'] = np.select(conditions_tipo_predio, choices_tipo_predio, default='SIN_ASIGNAR')

ofertas_final_v2.loc[ofertas_final_v2["FUENTE_PROCESAMIENTO"] != "R1_MATRICULA", 'CLASE_PREDIO'] = "SIN ASIGNAR"

ofertas_final_v2["DIVIPOLA_EN_R1"] = "NO"
ofertas_final_v2.loc[ofertas_final_v2["DIVIPOLA"].isin(df_R1_matricula["DIVIPOLA"].drop_duplicates().to_list()), "DIVIPOLA_EN_R1"] = "SI"

ofertas_final_v2.loc[(ofertas_final_v2["DIVIPOLA_EN_R1"] == "NO") & (ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "R1_MATRICULA"), "FUENTE_PROCESAMIENTO"] = "MUNICIPIO_SIN_R1"

ofertas_final_v2.reset_index(inplace = True)

Logical_validation_mun = ofertas_final_v2[(ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "Subdireccion") & (ofertas_final_v2["TIPO_GEORREF"].isin(["EXACTA", "APROX_OK"]))]["CODIGO_PREDIAL"].str[0:5] != ofertas_final_v2[(ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "Subdireccion") & (ofertas_final_v2["TIPO_GEORREF"].isin(["EXACTA", "APROX_OK"]))]["DIVIPOLA"]
ofertas_final_v2.loc[ofertas_final_v2["index"].isin(Logical_validation_mun[Logical_validation_mun == True].index), "TIPO_GEORREF"] = "APROX"

ofertas_final_v2.set_index("index")
ofertas_final_v2.drop(['Unnamed: 0',  'Direccion1', 'address',  'Consecutivo',  "SUBSTRING_22", 'SUBSTRING_67', "CODIGO_ANT_RURAL_APROX", "CODIGO_ANT_URBANO", "CODIGO_ANT_RURAL","CODIGO_ANT_URBANO_APROX"], axis = 1, inplace = True)
ofertas_final_v2["ENCONTRADO"] = "NO"
ofertas_final_v2.loc[(ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "R1_MATRICULA") | ((ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "Subdireccion") & (ofertas_final_v2["TIPO_GEORREF"].isin(["EXACTA", "APROX_OK"]))), "ENCONTRADO"] = "SI"



In [24]:
ofertas_final_v2[["ENCONTRADO", "FUENTE_PROCESAMIENTO","TIPO_GEORREF"]].value_counts(dropna=False)


ENCONTRADO  FUENTE_PROCESAMIENTO  TIPO_GEORREF
NO          Subdireccion          APROX           2676
            NaN                   NaN              403
SI          R1_MATRICULA          NaN              322
                                  EXACTA           133
                                  APROX_OK          96
            Subdireccion          EXACTA            28
            R1_MATRICULA          APROX             27
NO          MUNICIPIO_SIN_R1      APROX             22
SI          Subdireccion          APROX_OK          20
NO          MUNICIPIO_SIN_R1      NaN                9
Name: count, dtype: int64

In [25]:
ofertas_final_v2.loc[(ofertas_final_v2["DIVIPOLA_EN_R1"] == "NO") & (ofertas_final_v2["FUENTE_PROCESAMIENTO"] == "R1_MATRICULA"), "FUENTE_PROCESAMIENTO"] 

Series([], Name: FUENTE_PROCESAMIENTO, dtype: object)

In [37]:
# excel_writer = pd.ExcelWriter("../output/" + formatted_date + "_BASE_PROCESADA_AVALUOS_IGAC_SUCRE.xlsx", engine='xlsxwriter')

# ofertas_final_v2.to_excel(excel_writer, sheet_name= "Base_procesada", index = False)
# ofertas_final_v2.pivot_table(values = "Id", index=['DIVIPOLA', "Departamento_nombre", "Municipio_nombre"], columns=['ENCONTRADO'], aggfunc='count', fill_value=0, margins=True, margins_name='Total').to_excel(excel_writer, sheet_name= "Resumen_Encontrados")
# ofertas_final_v2[ofertas_final_v2["ENCONTRADO"] == "SI"].pivot_table(values = "Id", index=['DIVIPOLA', "Departamento_nombre", "Municipio_nombre"], columns=['ZONA'], aggfunc='count', fill_value=0, margins=True, margins_name='Total').to_excel(excel_writer, sheet_name= "Resumen_Zona")
# excel_writer.save()

with pd.ExcelWriter("../output/" + formatted_date + "_BASE_PROCESADA_AVALUOS_IGAC_SUCRE.xlsx") as writer:  
    ofertas_final_v2.to_excel(writer, sheet_name='Base_procesada', index = False)
    ofertas_final_v2.pivot_table(values = "Id", index=['DIVIPOLA', "Departamento_nombre", "Municipio_nombre"], columns=['ENCONTRADO'], aggfunc='count', fill_value=0, margins=True, margins_name='Total').to_excel(writer, sheet_name='Resumen_Encontrados')
    ofertas_final_v2[ofertas_final_v2["ENCONTRADO"] == "SI"].pivot_table(values = "Id", index=['DIVIPOLA', "Departamento_nombre", "Municipio_nombre"], columns=['ZONA'], aggfunc='count', fill_value=0, margins=True, margins_name='Total').to_excel(writer, sheet_name='Resumen_Zona')

In [31]:
pip install xlsxwriter


                                              0.0/153.0 kB ? eta -:--:--
     -------------------------------------- 153.0/153.0 kB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip
